In [1]:
# 환경 세팅
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

### Merge를 위한 데이터 불러오기 **(경로에 주의할 것)**

In [2]:
# 각 폴더명 딕셔너리
folder_name = {1:"1.회원정보", 2:"2.신용정보", 3:"3.승인매출정보", 4:"4.청구입금정보", 5:"5.잔액정보", 6:"6.채널정보", 7:"7.마케팅정보", 8:"8.성과정보"}

# 현재 디렉터리
HOME = os.getcwd()

# 폴더 리스트를 생성하고 폴더 경로들을 리스트로 저장하기
folder_list = []
for value in folder_name.values():
    folder_list.append(os.path.join(HOME, "data", "train", value))
print(len(folder_list))

# 폴더 내 파일 리스트 읽어오기
fileNameList = {}
for i in range(len(folder_list)):
    fileNameList[i+1] = os.listdir(folder_list[i])
print(fileNameList)

# 파일 경로 리스트 생성
filePathList = {}
temp = []
for i in range(len(folder_list)):
    for j in range(len(fileNameList[i+1])):
        temp.append(os.path.join(folder_list[i], fileNameList[i+1][j]))
    filePathList[i+1] = temp
    temp = []


print(filePathList)

8
{1: ['201807_train_회원정보.parquet', '201808_train_회원정보.parquet', '201809_train_회원정보.parquet', '201810_train_회원정보.parquet', '201811_train_회원정보.parquet', '201812_train_회원정보.parquet'], 2: ['201807_train_신용정보.parquet', '201808_train_신용정보.parquet', '201809_train_신용정보.parquet', '201810_train_신용정보.parquet', '201811_train_신용정보.parquet', '201812_train_신용정보.parquet'], 3: ['201807_train_승인매출정보.parquet', '201808_train_승인매출정보.parquet', '201809_train_승인매출정보.parquet', '201810_train_승인매출정보.parquet', '201811_train_승인매출정보.parquet', '201812_train_승인매출정보.parquet'], 4: ['201807_train_청구정보.parquet', '201808_train_청구정보.parquet', '201809_train_청구정보.parquet', '201810_train_청구정보.parquet', '201811_train_청구정보.parquet', '201812_train_청구정보.parquet'], 5: ['201807_train_잔액정보.parquet', '201808_train_잔액정보.parquet', '201809_train_잔액정보.parquet', '201810_train_잔액정보.parquet', '201811_train_잔액정보.parquet', '201812_train_잔액정보.parquet'], 6: ['201807_train_채널정보.parquet', '201808_train_채널정보.parquet', '201809_train_채널정보.parquet',

### 메모리 변환 및 병합을 돕는 함수

In [3]:
# 메모리를 32비트로 변환
def optimize_memory(df):
    for col in df.select_dtypes(include=['number']).columns:
        # 정수형은 가능한 가장 작은 타입으로 (int8, int16 등)
        if df[col].dtype == 'int64':
            df[col] = pd.to_numeric(df[col], downcast='integer')
        # 실수형은 float32로
        elif df[col].dtype == 'float64':
            df[col] = pd.to_numeric(df[col], downcast='float')
    return df

# 메인으로 파일을 받을 함수
def create_DataFrame(filePathList, month):
    if month >= 7& month <= 12:
        for i in range(8):

            # 단계 1 용량 줄이기 64bit -> 32bit
            df = pd.read_parquet(filePathList[i+1][month-7],engine = "fastparquet")
            df = optimize_memory(df)
            # 단계 2 병합
            if i == 0:
                temp_df = df.copy()
            else:
                temp_df = pd.merge(temp_df, df, on = ["기준년월","ID"], how = "left")


        return temp_df
    else:
        print("this is out of month")

### 함수를 실행시켜 월별로 저장

In [4]:
df1 = create_DataFrame(filePathList, 7)
df2 = create_DataFrame(filePathList, 8)
df3 = create_DataFrame(filePathList, 9)
df4 = create_DataFrame(filePathList, 10)
df5 = create_DataFrame(filePathList, 11)
df6 = create_DataFrame(filePathList, 12)

### parquet 파일로 데이터 용량 줄여서 저장하기

In [5]:
df1.to_parquet("credit_preprocessing_month_7.parquet")
df2.to_parquet("credit_preprocessing_month_8.parquet")
df3.to_parquet("credit_preprocessing_month_9.parquet")
df4.to_parquet("credit_preprocessing_month_10.parquet")
df5.to_parquet("credit_preprocessing_month_11.parquet")
df6.to_parquet("credit_preprocessing_month_12.parquet")

### 만들어진 파일의 정보 및 메모리 용량 확인

In [6]:
df1.info()

<class 'pandas.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Columns: 858 entries, 기준년월 to 혜택수혜율_B0M
dtypes: float32(57), float64(4), int16(152), int32(141), int8(454), object(50)
memory usage: 756.1+ MB


### 병합한 파일 저장한 것을 다시 불러와서 제대로 저장됬는지 확인하기

In [7]:
df_test= pd.read_parquet('credit_preprocessing_month_7.parquet', engine='fastparquet')
df_test.info()

<class 'pandas.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Columns: 858 entries, 기준년월 to 혜택수혜율_B0M
dtypes: float32(57), float64(4), int16(152), int32(141), int8(454), object(50)
memory usage: 756.1+ MB


### 1. 데이터 통합 (Concat) 및 시계열 정렬

#### 1단계: 데이터 수직 결합 (Concat) : 먼저 메모리 효율을 위해 리스트에 담아 한 번에 합치는 것이 좋음. <br>
#### pd.concat은 여러 개의 데이터프레임을 한 번에 합칠 때 효율적

In [8]:
# 1. 월별 데이터프레임을 리스트로 묶기
df_list = [df1, df2, df3, df4, df5, df6]
print

<function print(*args, sep=' ', end='\n', file=None, flush=False)>

In [10]:
# 2. 수직 결합 (위아래로 붙이기)
# ignore_index=True를 설정해야 기존의 의미 없는 인덱스를 버리고 새롭게 순차적인 인덱스를 부여합니다.
all_df = pd.concat(df_list, axis=0, ignore_index=True)

print(f"전체 행 개수: {len(all_df)}")

전체 행 개수: 2400000


### 2단계: 기준년월 형식 확인 및 변환
#### 정렬을 하기 전에 기준년월 컬럼이 숫자(202407)인지 문자열('2024-07')**인지, 혹은 날짜(datetime) 형식인지 확인 <br>
#### 만약 숫자로 되어 있다면 정렬에는 큰 문제가 없으나, 날짜 데이터 처리를 위해 datetime 객체로 바꾸는 것이 좋음

In [11]:
# 현재 데이터 타입 확인
print(all_df['기준년월'].dtype)

int32


In [12]:
# 만약 202307 같은 숫자/문자라면 datetime으로 변환 (선택 사항이지만 추천)
all_df['기준년월'] = pd.to_datetime(all_df['기준년월'], format='%Y%m')
print(all_df['기준년월'].dtype)

datetime64[us]


### 3단계: 시계열 정렬 (ID와 기준년월 기준)

#### 특정 사용자가 시간에 따라 어떻게 행동했는지 보려면 ID로 먼저 묶고, 그 안에서 월별로 정렬

In [13]:
# 3. ID와 기준년월 순으로 정렬
# ID를 1순위, 기준년월을 2순위로 정렬합니다.
all_df = all_df.sort_values(by=['ID', '기준년월'], ascending=[True, True])

In [14]:
# 인덱스 재설정
all_df = all_df.reset_index(drop=True)

In [15]:
# 결과 확인
all_df.head(10)

,기준년월,ID,남녀구분코드,연령,Segment,회원여부_이용가능,회원여부_이용가능_CA,회원여부_이용가능_카드론,소지여부_신용,소지카드수_유효_신용,...,변동률_RV일시불평잔,변동률_할부평잔,변동률_CA평잔,변동률_RVCA평잔,변동률_카드론평잔,변동률_잔액_B1M,변동률_잔액_일시불_B1M,변동률_잔액_CA_B1M,혜택수혜율_R3M,혜택수혜율_B0M
0,2018-07-01,TRAIN_000000,2,40대,D,1,1,0,1,1,...,0.999998,1.042805,0.999700,0.999998,0.999998,0.261886,0.270752,0.0,1.044401,1.280542
1,2018-08-01,TRAIN_000000,2,40대,D,1,1,0,1,1,...,0.999998,0.595262,1.000661,0.999998,0.999998,-0.140747,-0.117855,0.0,1.185349,1.555697
2,2018-09-01,TRAIN_000000,2,40대,D,1,1,0,1,1,...,0.999998,1.048667,1.000722,0.999998,0.999998,-0.057002,0.052830,0.0,0.879170,1.407221
3,2018-10-01,TRAIN_000000,2,40대,D,1,1,0,1,1,...,0.999998,0.595453,1.000911,0.999998,0.999998,-0.083067,0.044567,0.0,0.940814,1.587244
4,2018-11-01,TRAIN_000000,2,40대,D,1,1,0,1,1,...,0.999998,0.593415,1.000995,0.999998,0.999998,-0.038451,0.082848,0.0,0.937722,1.478965
5,2018-12-01,TRAIN_000000,2,40대,D,1,1,0,1,1,...,0.999998,0.591302,1.001020,0.999998,0.999998,-0.057150,0.061048,0.0,0.878859,1.398627
6,2018-07-01,TRAIN_000001,1,30대,E,1,1,1,1,1,...,1.092698,0.905663,0.999998,0.999998,0.999998,-0.563388,-0.670348,0.0,0.000000,0.000000
7,2018-08-01,TRAIN_000001,1,30대,E,1,1,1,1,1,...,1.070883,0.904136,0.999998,0.999998,0.999998,-0.028567,0.024387,0.0,0.000000,0.000000
8,2018-09-01,TRAIN_000001,1,30대,E,1,1,1,1,1,...,1.009655,0.903458,0.999998,0.999998,0.999998,-0.146730,-0.134340,0.0,0.000000,0.000000
9,2018-10-01,TRAIN_000001,1,30대,E,1,1,1,1,1,...,0.999068,0.902803,0.999998,0.999998,0.999998,0.021035,0.037035,0.0,0.000000,0.000000


### 4단계: 시계열 데이터가 제대로 합쳐졌는지 검증

#### 정렬 후에는 특정 ID의 데이터를 뽑아서 월별로 잘 나열되었는지 확인

In [16]:
# 샘플로 하나의 ID만 추출해서 흐름 확인
sample_id = all_df['ID'].iloc[0]
display(all_df[all_df['ID'] == sample_id][['ID', '기준년월']])

,ID,기준년월
0,TRAIN_000000,2018-07-01
1,TRAIN_000000,2018-08-01
2,TRAIN_000000,2018-09-01
3,TRAIN_000000,2018-10-01
4,TRAIN_000000,2018-11-01
5,TRAIN_000000,2018-12-01


#### 메모리 최적화

In [17]:
all_df = optimize_memory(all_df)

#### 불필요한 객체 삭제: 이제 개별 데이터(df1, df2 등)는 필요 없으므로 메모리에서 지워주는 것이 좋음.

In [18]:
import gc
del df1, df2, df3, df4, df5, df6, df_list
gc.collect() # 가비지 컬렉션 강제 실행

733

### 2. 탐색적 데이터 분석 (EDA) & 결측치 처리

#### 1단계: 결측치 상태 한눈에 파악하기 : isnull().sum()만 보는 것보다, 전체 데이터 대비 비율을 함께 보는 것이 의사결정에 도움 됨.

In [19]:
# 결측치 수와 비율을 확인하는 함수
def get_missing_info(df):
    missing_count = df.isnull().sum()
    missing_pct = (missing_count / len(df)) * 100
    missing_info = pd.concat([missing_count, missing_pct], axis=1, keys=['Missing Count', 'Missing %'])
    # 결측치가 있는 컬럼만 내림차순 정렬
    return missing_info[missing_info['Missing Count'] > 0].sort_values(by='Missing %', ascending=False)

missing_status = get_missing_info(all_df)
print(missing_status)

                Missing Count  Missing %
연체일자_B0M              2394336  99.764000
_3순위여유업종              2377725  99.071875
_3순위납부업종              2310187  96.257792
_2순위여유업종              2302286  95.928583
_3순위교통업종              2045455  85.227292
_2순위납부업종              2033640  84.735000
최종카드론_대출일자            1988330  82.847083
_1순위여유업종              1987260  82.802500
최종카드론_신청경로코드          1958226  81.592750
최종카드론_금융상환방식코드        1958126  81.588583
RV신청일자                1951236  81.301500
_2순위교통업종              1656423  69.017625
OS구분코드                1633566  68.065250
_3순위쇼핑업종              1312267  54.677792
_1순위납부업종              1216263  50.677625
_1순위교통업종              1164494  48.520583
_2순위쇼핑업종              1135042  47.293417
_3순위업종                1107898  46.162417
_2순위신용체크구분             958115  39.921458
_1순위쇼핑업종               922663  38.444292
_2순위업종                 912725  38.030208
혜택수혜율_B0M              555522  23.146750
_1순위업종                 539992  22.499667
최종유효년월_신용_이용    

In [20]:
# 샘플로 하나의 ID만 추출해서 흐름 확인
sample_id = all_df['ID'].iloc[0]
display(all_df[all_df['ID'] == sample_id][['ID', '기준년월', '연체일자_B0M']])

,ID,기준년월,연체일자_B0M
0,TRAIN_000000,2018-07-01,NaN
1,TRAIN_000000,2018-08-01,NaN
2,TRAIN_000000,2018-09-01,NaN
3,TRAIN_000000,2018-10-01,NaN
4,TRAIN_000000,2018-11-01,NaN
5,TRAIN_000000,2018-12-01,NaN


#### 2단계: 결측치 처리 전략 수립 (금융/카드 데이터 기준) : 금융 데이터에서 결측치는 단순한 데이터 누락이 아니라 "행동의 부재"를 의미하는 경우가 많음

#### 결측치 비율 80% 이상: 삭제 검토 (Drop)
#### 결측치 비율이 90%가 넘어가는 컬럼은 분석에 방해가 되므로 컬럼 자체를 삭제

In [21]:
# 결측치 80% 이상 컬럼 삭제
over_80 = missing_status[missing_status['Missing %'] >= 80].index
all_df.drop(columns=over_80, inplace=True)

In [22]:
# 결측치 수와 비율을 확인하는 함수
def get_missing_info(df):
    missing_count = df.isnull().sum()
    missing_pct = (missing_count / len(df)) * 100
    missing_info = pd.concat([missing_count, missing_pct], axis=1, keys=['Missing Count', 'Missing %'])
    # 결측치가 있는 컬럼만 내림차순 정렬
    return missing_info[missing_info['Missing Count'] > 0].sort_values(by='Missing %', ascending=False)

missing_status = get_missing_info(all_df)
print(missing_status)

                Missing Count  Missing %
_2순위교통업종              1656423  69.017625
OS구분코드                1633566  68.065250
_3순위쇼핑업종              1312267  54.677792
_1순위납부업종              1216263  50.677625
_1순위교통업종              1164494  48.520583
_2순위쇼핑업종              1135042  47.293417
_3순위업종                1107898  46.162417
_2순위신용체크구분             958115  39.921458
_1순위쇼핑업종               922663  38.444292
_2순위업종                 912725  38.030208
혜택수혜율_B0M              555522  23.146750
_1순위업종                 539992  22.499667
최종유효년월_신용_이용           534231  22.259625
혜택수혜율_R3M              488746  20.364417
가입통신회사코드               387570  16.148750
직장시도명                  244969  10.207042
최종유효년월_신용_이용가능         210447   8.768625
최종카드발급일자                41965   1.748542
RV전환가능여부                29473   1.228042
_1순위신용체크구분              27950   1.164583


#### 업종 및 쇼핑 관련 변수 (Fill with 'None' or 0)
#### _1순위쇼핑업종, _2순위업종 등 소비 패턴과 관련된 변수들
#### 결측치는 "해당 순위에 해당하는 업종 이용 실적이 없음"을 의미
#### 처리: 
* 범주형(문자열): 'Unknown' 또는 'None'으로 채워 하나의 카테고리로 만듬
* 수치형(금액/비율/실적/잔액/매출/승인금액, 이용금액, 잔액, 연체금액): 0으로 채움

In [23]:
# 샘플로 하나의 ID만 추출해서 흐름 확인
sample_id = all_df['ID'].iloc[0]
display(all_df[all_df['ID'] == sample_id][['ID', '기준년월', '_2순위교통업종']])

,ID,기준년월,_2순위교통업종
0,TRAIN_000000,2018-07-01,버스지하철
1,TRAIN_000000,2018-08-01,None
2,TRAIN_000000,2018-09-01,None
3,TRAIN_000000,2018-10-01,None
4,TRAIN_000000,2018-11-01,None
5,TRAIN_000000,2018-12-01,None


In [ ]:
# cols_to_zero = ['승인금액_컬럼명', '이용금액_컬럼명', '잔액_컬럼명'] # 실제 컬럼명으로 수정
# all_df[cols_to_zero] = all_df[cols_to_zero].fillna(0)

#### 최빈값이나 특정 카테고리로 채워야 하는 경우 (주로 속성 정보)

#### 직업코드, 주거형태, 신용등급 등: 정보가 없는 경우 가장 흔한 값으로 채우거나, 'Unknown' 혹은 '-1' 같은 별도의 범주를 만들어 관리

In [ ]:
# all_df['직업코드'] = all_df['직업코드'].fillna(all_df['직업코드'].mode()[0])
# # 또는
# all_df['신용등급'] = all_df['신용등급'].fillna(-1)

### 3단계: 탐색적 데이터 분석 (EDA) - "데이터와 대화하기"

#### 결측치를 처리하면서 동시에 데이터의 분포를 시각화하여 이상치나 특이점 찾기

#### ① 수치형 데이터 분포 확인 (히스토그램/박스플롯)

In [ ]:
# # 주요 수치형 컬럼 분포 확인 (예: 총 이용금액)
# plt.figure(figsize=(10, 5))
# sns.histplot(all_df['총_이용금액_컬럼명'], bins=50, kde=True)
# plt.title('Distribution of Usage Amount')
# plt.show()

# # 이상치 확인을 위한 박스플롯
# sns.boxplot(x=all_df['총_이용금액_컬럼명'])
# plt.show()

#### ② 범주형 데이터 확인 (성별, 지역 등)

In [ ]:
# # 성별이나 등급별 인원수 확인
# sns.countplot(data=all_df, x='성별_컬럼명')
# plt.show()

#### ③ 상관관계 분석 (Heatmap)

In [ ]:
# plt.figure(figsize=(12, 10))
# # 수치형 변수들만 선택하여 상관계수 산출
# sns.heatmap(all_df.select_dtypes(include=[np.number]).corr(), annot=False, cmap='coolwarm')
# plt.title('Correlation Heatmap')
# plt.show()

#### '기준년월' 별 데이터 흐름 확인하기 : 병합이 잘 되었는지 확인하기 위해 월별로 데이터 건수나 주요 지표의 평균이 일정한지 확인

In [ ]:
# 월별 데이터 건수 확인 (특정 월에 데이터가 쏠리거나 비어있지 않은지)
print(all_df.groupby('기준년월').size())

In [ ]:
# # 월별 평균 매출액 추이
# all_df.groupby('기준년월')['매출액_컬럼명'].mean().plot(kind='line', marker='o')
# plt.title('Monthly Average Sales')
# plt.show()